# Assignment 3
## Energy
You are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [ ]:
import pandas as pd
import numpy as np
import re

# Excel einlesen
df = pd.read_excel("Energy Indicators.xls")

# Lösche Spalten
del df['Unnamed: 0']
del df['Unnamed: 1']

# Spalten umbennen
columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
df.columns = columns
df_temp = df.loc[16:242]

# Convert Energy Supply to gigajoules
energy_suply = []
i = 16
while (i < 243):
    if(df_temp['Energy Supply'][i] == '...'):
        energy_suply.append(np.NaN)
    else:
        energy_suply.append(df_temp['Energy Supply'][i] * 1000000)
    i = i + 1

# Rename countries
df_temp = df_temp.replace({
    'Republic of Korea': 'South Korea', 
    'United States of America20': 'United States', 
    'United Kingdom of Great Britain and Northern Ireland19': 'United Kingdom',
    'China, Hong Kong Special Administrative Region3': 'Hong Kong'})

# Remove numbers and/or parenthesis from countries
countries = list(df_temp.Country)
i = 0
while (i < len(countries)):
    countries[i] = re.sub(r'\([^)]*\)', '', countries[i])
    countries[i] = re.sub(r'[0-9]', '', countries[i])
    i = i + 1

energy = pd.DataFrame()
energy['Country'] = countries
energy['Energy Supply'] = energy_suply
energy['Energy Supply per Capita'] = list(df_temp['Energy Supply per Capita'])
energy['% Renewable'] = list(df_temp['% Renewable'])

# CSV einlesen
GDP = pd.read_csv("world_bank.csv", skiprows=4)
GDP.rename(columns={'Country Name': 'Country'}, inplace=True)

# Rename countries
GDP = GDP.replace({
    'Korea, Rep.': 'South Korea', 
    'Iran, Islamic Rep.': 'Iran',
    'Hong Kong SAR, China': 'Hong Kong'})

# Excel einlesen
ScimEn = pd.read_excel("scimagojr.xlsx")

In [ ]:
def answer_one():   
    # Join --> inner join verwenden
    result = ScimEn.set_index('Country')[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index']].join(energy.set_index('Country')[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].join
                                          (GDP.set_index('Country')[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']], how='inner'), how='inner')    
    return result[result.Rank < 16]

answer_one()

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [ ]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [ ]:
# outer Join --> alle Datensätze werden genommen
def answer_two():
    result_inner = ScimEn.set_index('Country')[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index']].join(energy.set_index('Country')[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].join
                                          (GDP.set_index('Country')[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']], how='inner'), how='inner')    
    result_outer = ScimEn.set_index('Country')[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index']].join(energy.set_index('Country')[['Energy Supply', 'Energy Supply per Capita', '% Renewable']].join
                                          (GDP.set_index('Country')[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']], how='outer'), how='outer')    
    
    return len(result_outer)-len(result_inner)

answer_two()

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [ ]:
def Average(lst): 
    return sum(lst) / len(lst) 

def answer_three():
    Top15 = answer_one()
    avgGDP = pd.Series([])
    
    temp_Top15 = Top15[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    
    i = 0
    while (i < len(Top15)):
        country = temp_Top15.index[i]
        avg = Average(temp_Top15[temp_Top15.index == country].values[0])
        avgGDP[country] = avg
        i = i + 1
    return avgGDP.sort_values(ascending=False)

answer_three()

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [ ]:
# 2015 - 2006 nicht max - min
def answer_four():
    Top15 = answer_one()
    a = Top15[Top15.index == answer_three().index[5]][['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    return a.iloc[0]['2015']-a.iloc[0]['2006']

answer_four()

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [ ]:
def answer_five():
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].mean()

answer_five()

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [ ]:
def answer_six():
    Top15 = answer_one()
    a = Top15['% Renewable']
    return (a[a.values == max(list(a))].index[0], a[a.values == max(list(a))].values[0])

answer_six()

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [ ]:
def answer_seven():
    Top15 = answer_one()
    ratio = Top15['Self-citations']/Top15['Citations']
    Top15['Ratio-citations'] = ratio
    a = Top15[Top15['Ratio-citations'] == max(Top15['Ratio-citations'])].iloc[0]
    return (a.name, a['Ratio-citations'])

answer_seven()

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [ ]:
def answer_eight():
    Top15 = answer_one()
    population = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15['Estimate_Population'] = population
    return Top15.sort_values(by=['Estimate_Population'], ascending=False).iloc[2].name

answer_eight()

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [ ]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    a = Top15[['Citable docs per Capita', 'Energy Supply per Capita']]
    return a['Citable docs per Capita'].astype('float64').corr(a['Energy Supply per Capita'].astype('float64'))

answer_nine()

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    Top15 = answer_one()
    HighRenew = []
    for item in Top15['% Renewable']:
        if (item >= Top15['% Renewable'].median()):
            HighRenew.append(1)
        else:
            HighRenew.append(0)
        
    Top15['HighRenew'] = HighRenew
    Top15.sort_values(by=['Rank'], inplace=True)
    return Top15['HighRenew']

answer_ten()

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
def answer_eleven():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']    
    a = pd.DataFrame(columns = ['size', 'sum', 'mean', 'std'])
    
    for continent, values in Top15.groupby(ContinentDict):
        a.loc[continent] = [len(values['PopEst']), values['PopEst'].sum(), values['PopEst'].mean(), values['PopEst'].std()]
    
    return a

answer_eleven()

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    Top15['Continent'] = [ContinentDict[index] for index in Top15.index]
    Top15['bins'] = pd.cut(Top15['% Renewable'], 5)
    return Top15.groupby(['Continent','bins']).size()

answer_twelve()

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = (Top15['Energy Supply'] / Top15['Energy Supply per Capita']).astype(float)
    return Top15['PopEst'].apply(lambda x: '{:,}'.format(x))

answer_thirteen()

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!